In [1]:
import pandas as pd
import joblib

ImportError: Unable to import required dependencies:
numpy: No module named 'numpy'

## New Dataset
Using aggregated data set from Kaggle:
https://www.kaggle.com/datasets/spscientist/students-performance-in-exams


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [3]:
score_data_path = "datasets/student_score.csv"
df = pd.read_csv(score_data_path)

# Display the first few rows of the dataset and the column names to understand its structure
df.head()

,Unnamed: 0,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,0,female,NaN,bachelor's degree,standard,none,married,regularly,yes,3.0,school_bus,< 5,71,71,74
1,1,female,group C,some college,standard,NaN,married,sometimes,yes,0.0,NaN,5 - 10,69,90,88
2,2,female,group B,master's degree,standard,none,single,sometimes,yes,4.0,school_bus,< 5,87,93,91
3,3,male,group A,associate's degree,free/reduced,none,married,never,no,1.0,NaN,5 - 10,45,56,42
4,4,male,group C,some college,standard,none,married,sometimes,yes,0.0,school_bus,5 - 10,76,78,75


In [4]:
# Adding an average score column
df['AverageScore'] = df[['MathScore', 'ReadingScore', 'WritingScore']].mean(axis=1)

# Preparing the data with OneHotEncoder for categorical variables
categorical_features = ['Gender', 'EthnicGroup', 'ParentEduc', 'LunchType', 'TestPrep', 'ParentMaritalStatus', 'PracticeSport', 'IsFirstChild', 'WklyStudyHours']
numeric_features = ['MathScore', 'ReadingScore', 'WritingScore']  # dropped for a real prediction model, included here for data prep demonstration

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', 'passthrough', numeric_features)
    ])

# Splitting data into training and testing sets
X = df[categorical_features + numeric_features]
y = df['AverageScore']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# # For the purpose of saving preprocessor
# preprocessor.fit(X_train)

# X_train_transformed = preprocessor.transform(X_train)
# # Assuming you have X_test defined similarly
# X_test_transformed = preprocessor.transform(X_test)

# # To save
# joblib.dump(preprocessor, 'preprocessor.pkl')

# Creating a pipeline that includes preprocessing and a regression model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Fitting the model
model.fit(X_train, y_train)

# Predicting test set results
y_pred = model.predict(X_test)

# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R-squared:", r2)

RMSE: 2.1829618018382412e-14
R-squared: 1.0


## Try NN

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [7]:
# Adding an average score column
df['AverageScore'] = df[['MathScore', 'ReadingScore', 'WritingScore']].mean(axis=1)

# Preparing the data with OneHotEncoder for categorical variables
categorical_features = ['Gender', 'EthnicGroup', 'ParentEduc', 'LunchType', 'TestPrep', 'ParentMaritalStatus', 'PracticeSport', 'IsFirstChild', 'WklyStudyHours']
numeric_features = ['MathScore', 'ReadingScore', 'WritingScore']  # dropped for a real prediction model, included here for data prep demonstration

In [11]:

# Define the preprocessing for numeric and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', StandardScaler(), numeric_features)  # Scaling numeric features
    ])

# Splitting the data into training and testing sets
X = df[categorical_features + numeric_features]
y = df['AverageScore']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# For the purpose of saving preprocessor
preprocessor.fit(X_train)

X_train_transformed = preprocessor.transform(X_train)
# Assuming you have X_test defined similarly
X_test_transformed = preprocessor.transform(X_test)

# To save
joblib.dump(preprocessor, 'combined_generative/models_and_preprocessor/preprocessor.pkl')

# Fit the preprocessing pipeline on the training data and transform training and testing data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Neural network architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),  # Adjusted to match the input feature size
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2, verbose=1)

# Evaluate the model on the test set
test_loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Test MSE: {test_loss}')

# Model summary
model.summary()

Epoch 1/50


/opt/anaconda3/envs/microsoft_hackthon/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


613/613 ━━━━━━━━━━━━━━━━━━━━ 1s 489us/step - loss: 1143.0745 - val_loss: 0.5955
Epoch 2/50
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 421us/step - loss: 0.2587 - val_loss: 0.1127
Epoch 3/50
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step - loss: 0.0693 - val_loss: 0.0627
Epoch 4/50
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 401us/step - loss: 0.0446 - val_loss: 0.0426
Epoch 5/50
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 403us/step - loss: 0.0320 - val_loss: 0.0329
Epoch 6/50
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 393us/step - loss: 0.0234 - val_loss: 0.0285
Epoch 7/50
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step - loss: 0.0187 - val_loss: 0.0199
Epoch 8/50
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 392us/step - loss: 0.0147 - val_loss: 0.0183
Epoch 9/50
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 391us/step - loss: 0.0125 - val_loss: 0.0131
Epoch 10/50
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 388us/step - loss: 0.0108 - val_loss: 0.0242
Epoch 11/50
613/613 ━━━━━━━━━━━━━━━━━━━━ 0s 401us/step - loss: 0.0139 - val_loss: 0.0120
Epoch 12/50
613/613 ━━━━━━━━━━━━━━━━━━

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 128)            │         5,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 46,469 (181.52 KB)

 Trainable params: 15,489 (60.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 30,980 (121.02 KB)

In [10]:
model.save('combined_generative/models_and_preprocessor/average_score_model.h5')